1. Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам в словаре применяется только одна операция - удаление символа (1-n). Чтобы найти исправление из слова тоже удаляются символы и сравниваются с теми, что хранятся в словаре. Оцените качество полученного алгоритма теми же тремя метриками.

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
import gzip
import csv
import collections
import textdistance
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

from sklearn.metrics import classification_report, accuracy_score

In [2]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [4]:
mistakes = []
total = 0
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [5]:
corpus = open('corpus_5000.txt', 'w', encoding='utf-8')
with gzip.open('lenta-ru-news.csv.gz', 'rt', encoding='utf-8') as archive:
    reader = csv.reader(archive, delimiter=',', quotechar='"')
    for i, line in enumerate(reader):
        if i < 5000:
            corpus.write(line[2].replace('\xa0', ' ') + '\n')

In [6]:
def normalize(text):
    
    normalized_text = [(word.strip(punctuation)) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [7]:
corpus = []
for text in open('corpus_5000.txt', encoding='utf-8').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [8]:
vocab = set()

for sent in corpus:
    vocab.update(sent)


In [9]:
def predict_mistaken(word, vocab):

    if word in vocab:
        return 0
    else:
        return 1

In [10]:
y_true = []
y_pred = []

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] == pair[1]:
            y_true.append(0)
        else:
            y_true.append(1)
        
        y_pred.append(predict_mistaken(pair[1], vocab))

In [11]:
print(classification_report(y_true, y_pred, ))

             precision    recall  f1-score   support

          0       0.98      0.86      0.92      8707
          1       0.49      0.91      0.64      1303

avg / total       0.92      0.86      0.88     10010



In [12]:
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [13]:
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N

In [14]:
def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def known2(words):
    "Выбираем слова, которые есть в словаре forms"
    s = set()
    for w in words:
        s.update(forms.get(w,[]))
    return s

In [15]:
def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

In [16]:
forms = {}
for word in WORDS.keys():
    forms_keys = edits1(word)
    for form_key in forms_keys:
        if form_key in forms:
            forms[form_key].append(word)
        else:
            forms[form_key] = [word]

In [17]:
def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known2(edits1(word)) or [word])

def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

In [18]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[0]] = predicted
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [19]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.6927072927072927
0.2908672294704528
0.2471574595153325


2. Добавьте к полученному алгоритму исправления (Symspell) триграммную модель и проверьте, улучшает ли она качество. Триграммную модель нужно вставить туда, где у вас выбирается один из нескольких кандидатов на исправление.

In [27]:
newcorpus = [['<start>', '<start>'] + sent + ['<end>'] for sent in corpus]

In [28]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [29]:
unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

for sentence in newcorpus:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence))
    trigrams.update(ngrammer(sentence, n = 3))


In [30]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}
vec = CountVectorizer(analyzer='char', ngram_range=(1,1), min_df=10)
X = vec.fit_transform(vocab)

In [31]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

In [32]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)[0] #distance - чем больше, тем хуже, а similarity наоборот
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [33]:
def get_closest_hybrid_match(text, X, vec, topn=5, metric=textdistance.damerau_levenshtein):
    # ваш код здесь
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    sims = Counter()
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup,topn, metric=metric)

    
    return closest

In [34]:
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    word_pairs = [('<start>', '<start>')] + word_pairs
    pred_sent = []
    
    for j in range(1, len(word_pairs)):
        
        pred = None
        predicted = get_closest_hybrid_match(word_pairs[j][1], X, vec)
                
        prev_word1 = word_pairs[j-1][1]
        prev_word2 = word_pairs[j-2][1]
        
        if (prev_word2, prev_word1) not in bigrams:
            pred = predicted[0][0]
            
        else:
            
            lm_predicted = []
            for word, m in predicted:
                trigram = ' '.join([prev_word2, prev_word1, word])
                lm_predicted.append((word, (1+(trigrams[trigram]/bigrams[' '.join(prev_word2, prev_word1)]))))
            if lm_predicted:
                
                pred = sorted(lm_predicted, key=lambda x: -x[1])[0][0]
            
        
        if pred is None:
            pred = word_pairs[j][1]
        

        
        if pred == word_pairs[j][0]:
            correct += 1
        else:
            mistakes.append((word_pairs[j][0], word_pairs[j][1], pred))
        total += 1
            
        if word_pairs[j][0] == word_pairs[j][1]:
            total_correct += 1
            if word_pairs[j][0] !=  pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if word_pairs[j][0] == pred:
                mistaken_fixed += 1
    
    if not i % 50:
        print(i)
        print(correct/total)
            
                

0
0.6924688279301746
50
0.6989935095475496
100
0.7024087722451915
150
0.7075422851225406
200
0.7113905203077178
250
0.7172700856043351
300
0.7207977207977208
350
0.7236642325211482
400
0.7275329935060401
450
0.7304523970290344
500
0.7333376640249448
550
0.735693623443592
600
0.7387693240375871
650
0.7416125065870367
700
0.7435053885422576
750
0.7447113265755839
800
0.7461048348235798
850
0.7481731018398549
900
0.7505539887187752


In [35]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.7513986013986014
0.38871834228702995
0.19432640404272425
